In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

# ДЗ 1 BS

Сайт = https://echo-books.com/shop/

Код берет какое-то максимальное количество книг с каждой страницы категории книг. Можно делать больше/меньше если изменить переменную bookcount. Затем объединяет в одну таблицу, где у каждой уникальной книги может быть несколько категорий.

bookcount = 10, 36s, 106 уникальные книги

bookcount = 20, 45s, 203 уникальные книги

bookcount = 200, 1m44s, 1751 уникальные книги

bookcount = 2000, 10m45s, 8743 уникальные книги

In [33]:
bookcount = 2000

Отсюда берем категории

In [34]:
site = 'https://echo-books.com/shop/'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
data = {
    'ppp': bookcount
}
response = requests.post(site, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')


Получаем список ссылок на категории

In [35]:
links = []
all_categories = soup.find(class_="custom-category-menu")
for cat in all_categories.find_all("a"):
    link = cat.get('href')
    if link != '#': links.append(link)

Функция которая парсит каждую страницу.

In [48]:
def parse_page(soup):
    links = []
    prices = []
    availabilities = []
    titles = []
    authors = []
    categories = []
    title = soup.find(class_ = "woocommerce-products-header__title page-title")

    for book in soup.find(id="grid").find_all(class_ = "product"):
        a = book.find(class_ = "woocommerce-LoopProduct-link woocommerce-loop-product__link")
        if a:
            link = a.get('href')
            links.append(link)
            cur_tag = book.find(class_="woocommerce-Price-currencySymbol")
            if cur_tag:
                cur_tag.extract()
            prices.append(book.find('bdi').text.strip())
            availability = book.find(class_='custom-availability-label')
            availabilities.append(availability.text.strip()) if availability else availabilities.append(np.nan)
            titles.append(book.find(class_='woocommerce-loop-product__title').text.strip())
            author = book.find(class_='product-author')
            authors.append(author.text.strip()) if author else authors.append(np.nan)
            categories.append(title.text.strip())
        else: pass
    page_books = pd.DataFrame({
        'link': links,
        'price':prices,
        'availability':availabilities,
        'title':titles,
        'author':authors,
        'categories': categories
        })
    return page_books

In [37]:
all_books = pd.DataFrame({})

Проходит по каждой ссылке, парсит, создает одну большую таблицу с дубликатами

In [38]:
for link in links:
    site = link
    response = requests.post(site, data=data, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        page_books = parse_page(soup)
        all_books = pd.concat([all_books, page_books]).reset_index(drop=True)


In [39]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)
all_books

,link,price,availability,title,author,categories
0,https://echo-books.com/shop/moyo-nastoyashhee-...,"150,00",Нет в наличии,Моё настоящее имя. Истории с биографией (с пож...,Улицкая Л.Е.,Книги с пожеланием и автографом автора
1,https://echo-books.com/shop/bednaya-liza-i-ee-...,"150,00",В наличии,Бедная Лиза и ее родственники: антология русск...,"Екатерина Шульман, Галина Юзефович (составители)",Книги с пожеланием и автографом автора
2,https://echo-books.com/shop/lecz-20-vek-s-pozh...,"300,00",В наличии,Лец. 20 век (с пожеланием и автографом автора),Виктор Шендерович,Книги с пожеланием и автографом автора
3,https://echo-books.com/shop/sredi-gien-s-pozhe...,"150,00",В наличии,Среди Гиен (с пожеланием и автографом автора),Виктор Шендерович,Книги с пожеланием и автографом автора
4,https://echo-books.com/shop/russkoe-ekonomiche...,"150,00",В наличии,Русское Экономическое Чудо (с пожеланием и авт...,Сергей Алексашенко,Книги с пожеланием и автографом автора
...,...,...,...,...,...,...
9472,https://echo-books.com/shop/sketchbuk-anteikov...,"11,90",Нет в наличии,Скетчбук Anteikovich,Ярослава Мурашко,Бумажные изделия
9473,https://echo-books.com/shop/sejchas-bloknot-dl...,"16,53",В наличии,Сейчас. Блокнот для выхода из тупика,Елена Резанова,Бумажные изделия
9474,https://echo-books.com/shop/planing-dlya-tvorc...,"19,00",В наличии,Планинг для творческих людей. Необыкновенные п...,автора Н.,Бумажные изделия
9475,https://echo-books.com/shop/planing-dlya-tvorc...,"19,46",В наличии,Планинг для творческих людей. Изумительные сады,NaN,Бумажные изделия


Соединяет категории дубликатов по ссылкам

In [40]:
all_books['categories'] = all_books.groupby(['link'])['categories'].transform(lambda x : ', '.join(x)) 
  
all_books = all_books.drop_duplicates().reset_index(drop=True)


In [41]:
all_books

,link,price,availability,title,author,categories
0,https://echo-books.com/shop/moyo-nastoyashhee-...,"150,00",Нет в наличии,Моё настоящее имя. Истории с биографией (с пож...,Улицкая Л.Е.,"Книги с пожеланием и автографом автора, Художе..."
1,https://echo-books.com/shop/bednaya-liza-i-ee-...,"150,00",В наличии,Бедная Лиза и ее родственники: антология русск...,"Екатерина Шульман, Галина Юзефович (составители)","Книги с пожеланием и автографом автора, Художе..."
2,https://echo-books.com/shop/lecz-20-vek-s-pozh...,"300,00",В наличии,Лец. 20 век (с пожеланием и автографом автора),Виктор Шендерович,Книги с пожеланием и автографом автора
3,https://echo-books.com/shop/sredi-gien-s-pozhe...,"150,00",В наличии,Среди Гиен (с пожеланием и автографом автора),Виктор Шендерович,Книги с пожеланием и автографом автора
4,https://echo-books.com/shop/russkoe-ekonomiche...,"150,00",В наличии,Русское Экономическое Чудо (с пожеланием и авт...,Сергей Алексашенко,Книги с пожеланием и автографом автора
...,...,...,...,...,...,...
8738,https://echo-books.com/shop/sketchbuk-anteikov...,"11,90",Нет в наличии,Скетчбук Anteikovich,Ярослава Мурашко,Бумажные изделия
8739,https://echo-books.com/shop/sejchas-bloknot-dl...,"16,53",В наличии,Сейчас. Блокнот для выхода из тупика,Елена Резанова,Бумажные изделия
8740,https://echo-books.com/shop/planing-dlya-tvorc...,"19,00",В наличии,Планинг для творческих людей. Необыкновенные п...,автора Н.,Бумажные изделия
8741,https://echo-books.com/shop/planing-dlya-tvorc...,"19,46",В наличии,Планинг для творческих людей. Изумительные сады,NaN,Бумажные изделия


In [42]:
all_books.link.nunique()

8743

In [43]:
all_books.isnull().sum()

link              0
price             0
availability      2
title             0
author          470
categories        0
dtype: int64

In [44]:
all_books.groupby('categories').count()

,link,price,availability,title,author
categories,,,,,
Бумажные изделия,100,100,100,100,32
"Выбор редакции, Новинки, Искусство, Прикладная литература",1,1,1,1,1
"Выбор редакции, Новинки, Книги без цензуры, Прикладная литература",1,1,1,1,1
"Выбор редакции, Новинки, Книги без цензуры, Публицистика, Прикладная литература",1,1,1,1,1
"Выбор редакции, Новинки, Книги без цензуры, Художественная литература",1,1,1,1,1
...,...,...,...,...,...
"Художественная литература, Прикладная литература",24,24,24,24,23
"Художественная литература, Публицистика",41,41,41,41,41
"Художественная литература, Публицистика, Прикладная литература",11,11,11,11,11


In [45]:
all_books.to_csv('all_books.csv')

# Альтернативный вариант

Если делать без категорий, можно собрать огромную таблицу всех книг. 

bookcount = 2000, 2m 30s, 2000 книг

bookcount = -1, 11m, много книг

In [51]:
bookcount = -1
data = {
    'ppp': bookcount
}
site = 'https://echo-books.com/shop/'
response = requests.post(site, data=data, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    no_categories_all_books = parse_page(soup)

In [55]:
no_categories_all_books = no_categories_all_books.drop('categories', axis=1)

In [56]:
no_categories_all_books.to_csv('no_categories_all_books.csv')